In [15]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline




# Build the Neural Network

神经网络由对数据执行操作的层/模块组成。
[torch.nn](https://pytorch.org/docs/stable/nn.html) 命名空间提供了构建您自己的神经网络所需的所有构建块。
PyTorch 中的每个模块都是nn.Module 的子类。神经网络本身就是一个模块，由其他模块（层）组成。
这种套结构允许轻松构建和管理复杂的架构。

在以下部分中，我们将构建一个神经网络来对 FashionMNIST 数据集中的图像进行分类。


In [16]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 获取训练设备
我们希望能够在 GPU 或 MPS 等硬件加速器（如果可用）上训练我们的模型。
让我们检查一下torch.cuda 或torch.backends.mps是否可用，否则我们使用 CPU。
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html)
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)


In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## 定义类

我们通过子类化来定义我们的神经网络``nn.Module``，并在``__init__``中初始化神经网络层。
每个nn.Module子类都在方法中重写forward方法实现对输入数据的操作。


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建 的一个实例NeuralNetwork，并将其移动到device，并打印其结构。

In [19]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用该模型，输入数据传递给它。这将执行模型的forward以及一些后台操作[background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)。
不要直接调用model.forward()

模型会返回一个二维张量，其中 dim=0 对应于每个类别的 10 个原始预测值的每个输出，dim=1 对应于每个输出的各个值。
我们通过nn.Softmax实例预测概率。



In [20]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


--------------




## 模型层

让我们分解 FashionMNIST 模型中的各个层。为了说明这一点，我们将采用 3 张大小为 28x28 的图像的小批量样本，看看当我们将其传递到网络时会发生什么。



In [21]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
我们初始化nn.Flatten 层 [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
将每个2D 28x28 图像转换为 784 个像素值的连续数组（维持小批量维度（在 dim=0 时））。



In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
线性层 [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
线性层 是一个使用其存储的权重和偏差对输入应用线性变换的模块。


In [23]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations是在模型的输入和输出之间创建复杂映射的原因。它们在线性变换后应用以引入非线性，帮助神经网络学习各种现象。
在此模型中，我们在线性层之间使用[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)，但还有其他activations可以在模型中引入非线性


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3074,  0.0675, -0.0774,  0.0334, -0.0727,  0.2357,  0.6654, -0.1711,
          0.5300,  0.1697, -0.2302,  0.1180,  0.4438, -0.1377,  0.0037, -0.1898,
         -0.0690,  0.3193, -0.5916, -0.2610],
        [ 0.0395,  0.0617,  0.1823,  0.0327,  0.2647, -0.0372,  0.8493, -0.0528,
          0.3781,  0.0273, -0.3975,  0.0993,  0.2535, -0.0742, -0.2080,  0.2996,
         -0.0400,  0.1853, -0.2708, -0.1649],
        [-0.0618, -0.0820,  0.3211,  0.1321,  0.3996,  0.0713,  0.8619, -0.2154,
          0.1736,  0.2574, -0.8609, -0.2147,  0.3523, -0.2258, -0.0046, -0.1344,
         -0.1148,  0.6483, -0.6095, -0.1567]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3074, 0.0675, 0.0000, 0.0334, 0.0000, 0.2357, 0.6654, 0.0000, 0.5300,
         0.1697, 0.0000, 0.1180, 0.4438, 0.0000, 0.0037, 0.0000, 0.0000, 0.3193,
         0.0000, 0.0000],
        [0.0395, 0.0617, 0.1823, 0.0327, 0.2647, 0.0000, 0.8493, 0.0000, 0.3781,
         0.0273, 0.0000, 0.0993, 0.2535, 0.0000, 0.00

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) 是模块的有序容器。
数据按照定义的相同顺序传递通过所有模块。您可以使用顺序容器来组合一个快速网络，例如seq_modules.


In [25]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.

神经网络的最后一个线性层返回logits -原始值在区间[-infty, infty}，被传递到 nn.Softmax模块。
Logits 缩放为值 [0, 1]，表示模型对每个类别的预测概率。
dim参数指示维度，沿该维度值的总和必须为 1。



In [26]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 模型参数

神经网络内的许多层都是参数化的，即具有在训练期间优化的相关权重和偏差。
子类nn.Module会自动跟踪模型对象中定义的所有字段，并使所有参数都可以使用模型parameters()或named_parameters()方法进行访问。

在此示例中，我们迭代每个参数，并打印其大小及其值的预览。

In [27]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0048,  0.0315, -0.0130,  ..., -0.0289, -0.0353,  0.0141],
        [ 0.0242, -0.0066, -0.0319,  ...,  0.0299, -0.0068,  0.0126]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0289,  0.0098], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0077,  0.0353, -0.0248,  ...,  0.0126,  0.0215, -0.0017],
        [ 0.0229, -0.0149, -0.0011,  ...,  0.0330,  0.0156,  0.0096]],
       device='cuda:0', grad_fn=<Sl

--------------




## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

